In [252]:
# Importa bibliotecas
import numpy as np
import matplotlib.pyplot as plt
from math import *
from scipy.integrate import odeint

In [253]:
def set_axes_equal(ax):
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)
    
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([0, z_middle + plot_radius])

In [254]:
d_ar = 1.2
g = 9.8
r = (0.66/(2*pi))
m = 0.27
cl = 1
A = pi*(r**2)
Cd = 0.077

quadra_x = [0, 18, 18, 0, 0]
quadra_y = [-4.5, -4.5, 4.5, 4.5, -4.5]
quadra_z = [0, 0, 0, 0, 0]

rede_x = [9, 9, 9, 9, 9]
rede_y = [-4.5, -4.5, 4.5, 4.5, -4.5]
rede_z = [1.43, 2.43, 2.43, 1.43, 1.43]

In [255]:
angulo_vertical = radians(0)
angulo_horizontal = radians(0)

x0 = 0
y0 = 0
z0 = 2.5

v0 = 126.3/3.6
vx0 = v0*cos(angulo_vertical)*cos(angulo_horizontal)
vy0 = 0
vz0 = -v0*sin(angulo_vertical)

w = 4*(2*pi)
wx = w*sin(angulo_vertical)*cos(90-angulo_horizontal)
wy = 0
wz = w*cos(angulo_vertical)

w = np.array([wx, wy, wz])

info_0 = [x0, y0, z0, vx0, vy0, vz0]

In [256]:
dt = 1e-3
t_lista = np.arange(0,1+dt,dt)
i = 0
def modelo(info, t_lista):
    
    x = info[0]
    y = info[1]
    z = info[2]
    vx = info[3]
    vy = info[4]
    vz = info[5]
    V_vetor = np.array([vx, vy, vz])
    
    v_xy = sqrt(vx**2 + vy**2)
    v = sqrt(v_xy **2 + vz**2)

    cos_vertical = v_xy/v
    sen_vertical = -vz/v
    cos_horizontal = vx/v_xy
    sen_horizontal = vy/v_xy
    
    Fa = 0.5*d_ar*A*Cd*(v**2)
    Fa_x = -Fa*cos_vertical*cos_horizontal
    Fa_y = -Fa*cos_vertical*sen_horizontal
    Fa_z = Fa*sen_vertical
    
    Fmag = cl*pi*d_ar*(r**3)*np.cross(w, V_vetor)
    Fmag_x = Fmag[0]
    Fmag_y = Fmag[1]
    Fmag_z = Fmag[2]
    
    dVx_dt = (Fa_x + Fmag_x)/m
    dVy_dt = (Fa_y + Fmag_y)/m
    dVz_dt = (-(m*g) + Fa_z + Fmag_z)/m
    
    if z <= 0:
        return [0, 0, 0, 0, 0, 0]
    else:
        return [vx, vy, vz, dVx_dt ,dVy_dt ,dVz_dt]

In [257]:
infos_Odeint = odeint(modelo, info_0, t_lista)

posX = list(infos_Odeint[:,0])
posY = list(infos_Odeint[:,1])
posZ = list(infos_Odeint[:,2])

get_ipython().magic('matplotlib qt5')
import matplotlib.patches as patches
from matplotlib import animation
%matplotlib qt5

# Defindo o tempo decorrido para a bola bater na rede
# Dado: 0.267 segundos
for i, t in enumerate(t_lista):
    if posX[i] >= 9:
        tempo = t
        break 

ax = plt.axes(projection='3d')
ax.plot3D(posX, posY, posZ, color='orange', label=f"{tempo} segundos")
ax.plot3D(rede_x, rede_y, rede_z, color='grey')
ax.plot3D(quadra_x, quadra_y, quadra_z, color='black')
ax.plot3D(posX[0], posY[0], posZ[0], 'ro') # Bola
plt.title("Trajetória da Bolinha de Tênis")
plt.grid()
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.legend()
set_axes_equal(ax)
plt.show()

